In [ ]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=0d924ace548b1d8169613563e6ed14318eff6884b01e976552015c055bfc5135
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [6]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [7]:
lr = LinearRegression(featuresCol='features',labelCol='label',
                      predictionCol='prediction')

In [8]:
lrModel = lr.fit(training)

In [9]:
lrModel.coefficients #feature importance related with label

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [10]:
lrModel.intercept

0.14228558260358093

In [12]:
training_summary = lrModel.summary

In [13]:
training_summary.rootMeanSquaredError

10.16309157133015

In [23]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [17]:
all_data = all_data.randomSplit([0.7,0.3])

In [18]:
all_data

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [26]:
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [28]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                354|
|   mean|0.49270669077575774|
| stddev| 10.016896465630252|
|    min|-28.046018037776633|
|    max|  27.78383192005107|
+-------+-------------------+



In [29]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                147|
|   mean|-0.3109991062239958|
| stddev| 11.023910075598282|
|    min|-28.571478869743427|
|    max| 27.111027963108548|
+-------+-------------------+



In [30]:
correct_model = lr.fit(train_data) #train the data

In [31]:
test_results = correct_model.evaluate(test_data)

In [32]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-28.978913309047446|
|-28.010401934014144|
| -24.99786443054734|
| -21.12422752785841|
|-15.712376838485469|
|-17.205835962548633|
|-17.217846171207498|
|-15.509536919333755|
|-16.445029588624163|
|-15.241185843293097|
|-17.567937619000244|
|-13.549448134554911|
|-15.658753166888113|
| -16.17853579983377|
|-12.712209205346744|
| -18.16126893055735|
|-13.908798751585659|
|-13.374247963742329|
|-14.163653803614498|
| -13.82427270674053|
+-------------------+
only showing top 20 rows



In [33]:
test_results.rootMeanSquaredError

10.939626319229795

In [34]:
unlabeled_data = test_data.select("features")

In [35]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [36]:
predictions = correct_model.transform(unlabeled_data)

In [37]:
predictions.show() #predicted value for the given feature

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  0.4074344393040177|
|(10,[0,1,2,3,4,5,...|   1.204918505531071|
|(10,[0,1,2,3,4,5,...| -1.7383427520543833|
|(10,[0,1,2,3,4,5,...|  1.2396667535849852|
|(10,[0,1,2,3,4,5,...| -1.7818235183978743|
|(10,[0,1,2,3,4,5,...| 0.14043633667261918|
|(10,[0,1,2,3,4,5,...|  0.5256391498963926|
|(10,[0,1,2,3,4,5,...| -0.5761221216877361|
|(10,[0,1,2,3,4,5,...|  0.7129413163849174|
|(10,[0,1,2,3,4,5,...|-0.19619895013811928|
|(10,[0,1,2,3,4,5,...|  2.2083927391675675|
|(10,[0,1,2,3,4,5,...| -1.7994230208243425|
|(10,[0,1,2,3,4,5,...|  0.8366002571369244|
|(10,[0,1,2,3,4,5,...|  1.8495572907583293|
|(10,[0,1,2,3,4,5,...| -1.1548786898120245|
|(10,[0,1,2,3,4,5,...|   4.388827368854478|
|(10,[0,1,2,3,4,5,...|  0.4882039756949008|
|(10,[0,1,2,3,4,5,...| 0.39639923835022617|
|(10,[0,1,2,3,4,5,...|  1.6628800182594428|
|(10,[0,1,2,3,4,5,...|   1.33283